# Libraries

In [65]:
# imports
import pandas as pd
import numpy as np
import argparse

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

# Import models from another notebook
%run model.ipynb
%run util.ipynb

In [2]:
# # params
# parser = argparse.ArgumentParser(description='Process inputs for training a model')
# parser.add_argument('--model_id', default='dense_model')
# parser.add_argument('--dropout', default='0.4', type=float)
# parser.add_argument('--layers_count', default='2', type=int)
# parser.add_argument('--dense_units', default='64', type=int)
# parser.add_argument('--hidden_activation', default='relu')

# args = parser.parse_args()
# print("Args: " + str(args))

# Preprocessing

### Load sudoku

In [2]:
# load inputs
sudokus = pd.read_csv('./datasets/sudoku-kaggle.csv')
print("Full shape:", sudokus.shape)
subset = sudokus.sample(n=50000).values
print("Subset shape:", subset.shape)

Full shape: (1000000, 2)
Subset shape: (50000, 2)


### Split into data and labels

In [3]:
# Split into puzzles and solutions
puzzles, solutions = subset[:, 0], subset[:, 1]
print("Number of puzzles:", len(puzzles))
print("Number of solutions:", len(solutions))

Number of puzzles: 50000
Number of solutions: 50000


### Reshape the data

In [4]:
# Reshape to 9x9
reshape_f = lambda x: np.reshape([int(digit) for digit in x], (9, 9))
reshaped_puzzles = np.array(list(map(reshape_f, puzzles)))
reshaped_solutions = np.array(list(map(reshape_f, solutions)))
print("Shape of puzzles:", reshaped_puzzles.shape)
print("Shape of solutions:", reshaped_solutions.shape)

Shape of puzzles: (50000, 9, 9)
Shape of solutions: (50000, 9, 9)


In [5]:
# To one-hot encoding
one_hot_puzzles = to_one_hot(reshaped_puzzles)
one_hot_solutions = to_one_hot(reshaped_solutions - 1)
print("Shape of puzzles:", one_hot_puzzles.shape)
print("Shape of solutions:", one_hot_solutions.shape)

Shape of puzzles: (50000, 9, 9, 10)
Shape of solutions: (50000, 9, 9, 9)


### Train test split

In [6]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(one_hot_puzzles, one_hot_solutions, 
                                                    test_size=0.1, random_state=42)
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing data shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

Training data shape: (45000, 9, 9, 10)
Training labels shape: (45000, 9, 9, 9)
Testing data shape: (5000, 9, 9, 10)
Testing labels shape: (5000, 9, 9, 9)


In [7]:
# validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Validating data shape:", X_val.shape)
print("Validating labels shape:", y_val.shape)

Training data shape: (40500, 9, 9, 10)
Training labels shape: (40500, 9, 9, 9)
Validating data shape: (4500, 9, 9, 10)
Validating labels shape: (4500, 9, 9, 9)


In [69]:
# Create new X_train from y_train
X_train_completed = to_puzzles(y_train)
X_val_completed = to_puzzles(y_val)
print("Training data shape:", X_train_completed.shape)
print("Training labels shape:", y_train.shape)
print("Validating data shape:", X_val_completed.shape)
print("Validating labels shape:", y_val.shape)

Training data shape: (40500, 9, 9, 10)
Training labels shape: (40500, 9, 9, 9)
Validating data shape: (4500, 9, 9, 10)
Validating labels shape: (4500, 9, 9, 9)


# Start training

In [71]:
# initialize model
model = get_model(X_train.shape[1:], model_id='dense_model')
print("Model input shape:", model.input.shape)
print("Model output shape:", model.output.shape)

Model input shape: (?, 9, 9, 10)
Model output shape: (?, 9, 9, 9)


In [72]:
%%time
# train model
model.fit(delete_digits(X_train_completed, 0), y_train, batch_size=128, epochs=1, verbose=1)

Epoch 1/1
40500/40500 [==============================] - 43s 1ms/step - loss: 0.4293 - acc: 0.9332
Wall time: 43.4 s


In [ ]:
%%time
early_stop = EarlyStopping(patience=2, verbose=1, restore_best_weights=True)

i = 1
for nb_epochs, nb_delete in zip(
        [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  # epochs for each round
        [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55]  # digit to pull off
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    model.fit(
        delete_digits(X_train_completed, nb_delete),  # delete digits from training sample
        y_train,
        validation_data=(
            delete_digits(X_val_completed, nb_delete), # delete same amount of digit from validation sample
            y_val),
        batch_size=128,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )

Pass n° 1 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 34s 837us/step - loss: 0.0415 - acc: 0.9884 - val_loss: 0.0272 - val_acc: 0.9890
Pass n° 2 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 27s 678us/step - loss: 0.0593 - acc: 0.9778 - val_loss: 0.0540 - val_acc: 0.9783
Epoch 2/2
21376/40500 [==============>...............] - ETA: 12s - loss: 0.0575 - acc: 0.9779

In [282]:
%%time
predictions = list(map(from_one_hot, iterative_predict(X_test[:100], model)))
# deltas = total_diff(from_one_hot(y_test) + 1, predictions)
deltas = [correct_solution(from_one_hot(puzzle), solution) for puzzle, solution in zip(X_test[:100], predictions)]
accuracy = np.mean(deltas)

print(
"""
Grid solved:\t {}
Correct ones:\t {}
Accuracy:\t {}
""".format(len(deltas), np.sum(deltas), accuracy))


Grid solved:	 100
Correct ones:	 0
Accuracy:	 0.0

Wall time: 13 s
